Questão nro. 1 (2 pontos):
Implemente um programa em C++ que gere n números aleatórios inteiros (entre 1 e 999), populando
um vetor. Fazendo uso de OpenMP, calcule a média e desvio padrão dos elementos deste vetor. Você deve
testar para n grandes, produzindo um gráfico onde X é o tamanho de n e Y é o tempo de execução. Teste
também seu programa para rodar com 2, 3, 5, 7 e 9 threads, avaliando o desempenho.
Entregue o código-fonte, o gráfico gerado e comentários sobre o desempenho conforme o número de
threads foi sendo modificado.

In [ ]:
%%writefile media_desvio.cpp
#include <iostream>
#include <stdlib.h>
#include <chrono>
#include <random>
#include <vector>
#include <omp.h>
using namespace std;

int main(int argc, char* argv[]) {
    int n = atoi(argv[1]);

    unsigned seed = chrono::system_clock::now().time_since_epoch().count();
    default_random_engine generator(42);

    uniform_int_distribution<int> gerador(1, 999);

    vector<int> numeros_aleatorios(n);

    omp_set_num_threads(9);

    for (int i = 0; i < n; i++) {
        numeros_aleatorios[i] = gerador(generator);
        //cout << "Aqui: " << numeros_aleatorios[i] << endl;
    }


    int total_sum = 0;
    //#pragma omp parallel for num_threads(4) reduction(+ : total_sum)
    #pragma omp parallel for reduction(+ : total_sum)
    for (int i = 0; i < n; i++){
      total_sum += numeros_aleatorios[i];
    }

    double media = total_sum/n;

    double soma_local_dp = 0;
    #pragma omp parallel for reduction(+ : soma_local_dp)
    for (int i = 0; i < n; i++){
      double elemento = numeros_aleatorios[i] - media;
      soma_local_dp += pow(elemento,2);
    }    

    double std = std::sqrt(soma_local_dp/n);
    cout << "Desvio: " << std;

    return 0;
}

Writing media_desvio.cpp


In [ ]:
!g++ -Wall -O3 -fopenmp -g media_desvio.cpp -o media_desvio

media_desvio.cpp: In function ‘int main(int, char**)’:
media_desvio.cpp:12:14: warning: unused variable ‘seed’ [-Wunused-variable]
   12 |     unsigned seed = chrono::system_clock::now().time_since_epoch().count();
      |              ^~~~


In [ ]:
!./media_desvio 100000000000

Desvio: 578.068

In [ ]:
#35 578.068 sem openmp
#32 segundos 100000000000 com open

#com 9 threads

Questão nro. 2 (4 pontos): Em nossas aulas tivemos a oportunidade de executar o problema Saxpy com a biblioteca Thrust. Você pode encontrar o código executado em aula neste endereço: https://insper.github.io/supercomp/aulas/16-gpu-customizacao/.

Sua tarefa:

1. Implemente uma versão do Saxpy com OpenMP (8 threads por padrão). [2 pontos]

2. Execute a versão com OpenMP e a versão em GPU. Compare os tempos de execução. No caso da GPU ser mais rápida, busque aumentar o número de threads do OpenMP (até um limite de 32). O tempo de execução do OpenMP melhorou? Justifique também caso OpenMP esteja sendo mais rápido que GPU. Há algo relacionado a movimentação de dados entre CPU e GPU? [2 pontos]

In [ ]:
%%writefile saxpy.cpp
#include <iostream>
#include <stdlib.h>
#include <chrono>
#include <random>
#include <vector>
#include <omp.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iomanip>
using namespace std;

int main(int argc, char* argv[]) {
    auto t_start = std::chrono::high_resolution_clock::now();
    int n = atoi(argv[1]);
    int m = atoi(argv[2]);

    unsigned seed = chrono::system_clock::now().time_since_epoch().count();
    default_random_engine generator(42);

    uniform_int_distribution<int> gerador(1, 999);

    vector<int> a(n, 0);
    vector<int> b(n, 0);
    vector<int> c(n, 0);


    omp_set_num_threads(4);

    #pragma omp parallel
    for (int i = 0; i < n; i++) {
        a[i] = rand(); //gerador(generator);
        b[i] = rand();
        //cout << "Aqui1: " << a[i] << endl;
        //cout << "Aqui2: " << b[i] << endl;
    }

    int sum = 0;
    #pragma omp parallel for reduction(+ : sum)
    // pragma omp parallel for shared(c)
    for (int i = 0; i < n; i++){
      sum = m*(a[i]+b[i]);
      c[i] = sum;
    }

    for (int i = 0; i < n; i++ ){
         cout << setw(6) << c[i] << " = " 
          << setw(2) << m
          << "*" << setw(5) << a[i]
          << "+" << setw(5) << b[i]
          << endl;

    }

    auto t_end = std::chrono::high_resolution_clock::now();

    double elapsed_time_ms = std::chrono::duration<double, std::milli>(t_end-t_start).count();
    double elapsed_time_s = elapsed_time_ms/1000;
    std::cout << "Tempo de execução: " << elapsed_time_s << " s" << std::endl;

    return 0;
}

Overwriting saxpy.cpp


In [ ]:
!g++ -Wall -O3 -fopenmp -g saxpy.cpp -o saxpy_cpp

saxpy.cpp: In function ‘int main(int, char**)’:
saxpy.cpp:17:14: warning: unused variable ‘seed’ [-Wunused-variable]
   17 |     unsigned seed = chrono::system_clock::now().time_since_epoch().count();
      |              ^~~~


In [ ]:
!./saxpy_cpp 3 10

-1104420104 = 10*304089172+1303455736
1271038494 = 10*35005211+521595368
-1258246520 = 10*294702567+1726956429
Tempo de execução: 0.0033827 s


Esperado:

```
Aqui: 1
Aqui: 525
Aqui: 735
Aqui: 264
Aqui: 376
Aqui: 197
  1578 =  3*    1+  525
  2997 =  3*  735+  264
  1719 =  3*  376+  197
```



In [ ]:
#include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>

 using namespace  std;

struct saxpy
{
    int a;    
    saxpy(int a_) : a(a_) {};
    __host__ __device__
    double operator()(const int& x, const int& y) {
           return a * x + y;
    }
};

int main(int argc, char* argv[]) {
     if (argc != 3) {
         cerr <<
          "***Numero incorreto de argumentos ***\n";
         return 1;
     }

     int n = atoi(argv[1]);
     int m = atoi(argv[2]);

     //gerar numeros aleatorios
     thrust::host_vector<int> a(n);
     thrust::host_vector<int> b(n);
     thrust::host_vector<int> c(n);
     thrust::generate(a.begin(), a.end(), rand);
     thrust::generate(b.begin(), b.end(), rand);

     //transferimos para a GPU
     thrust::device_vector<int> d_a = a;
     thrust::device_vector<int> d_b = b;

     //transformacao

     thrust::transform(d_a.begin(), d_a.end(),
                       d_b.begin(), d_b.end(),
                       saxpy(m));

     thrust::copy(d_b.begin(), d_b.end(),
     c.begin()); 

     for (int i = 0; i < n; i++ )
         cout << setw(6) << c[i] << " = " 
          << setw(2) << m
          << "*" << setw(5) << a[i]
          << "+" << setw(5) << b[i]
          << endl;

}

Overwriting saxpy_cu.cu


In [ ]:
!nvcc saxpy_cu.cu -o saxpy_cu

In [ ]:
!./saxpy_cu 3 10

1714636915 = 10*1804289383+1714636915
1957747793 = 10*846930886+1957747793
424238335 = 10*1681692777+424238335
Tempo de execução: 0.224654 s


Questão nro. 3 (2 pontos): Implemente a versão do código abaixo com Thrust. Atente-se ao tratamento
adequado para a geração de números aleatórios. Você deve usar, obrigatoriamente, a implementação
transform-reduce da biblioteca Thrust, disponı́vel em https://tinyurl.com/3namnuhv


critical - varias threads tentando acessar a posição 2

In [ ]:
 %%writefile pi.cu
#include <chrono>
#include <random>
#include <vector>
#include <cstdlib>
#include <algorithm>
#include <iostream>
#include <iomanip>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/functional.h>
#include <thrust/copy.h>

struct square_sum {
    // não tem parametro interno, então não precisa nem de construtor de copia
    __host__ __device__
    double operator()(const double &x, const double &y)  { // não é o x vetor, é a coordenada do vetor, só um ponto
        return x * x + y * y;
    }
};

struct greater_than_one {
    // não tem parametro interno, então não precisa nem de construtor de copia
    __host__ __device__
    double operator()(double num)  { // não é o x vetor, é a coordenada do vetor, só um ponto
        return num <= 1 ? 1 : 0;
    }
};

int main(){
  long N = 100000000L;
  long sum = 0;

  thrust::host_vector<double> x(N);
  thrust::host_vector<double> y(N);
  thrust::generate(x.begin(), x.end(), rand);
  thrust::generate(y.begin(), y.end(), rand);

  thrust::transform(x.begin(), x.end(), y.begin(), x.begin(),square_sum()); //passa incio do x e end dele, o y, mas o resultado vai ser armazenado no x
  double pi = 4.0*thrust::transform_reduce(x.begin(), x.end(), greater_than_one(), 0, thrust::plus<double>())/(double)N;
  std::cout << "Pi is: " << pi << std::endl;

  return 0;
}



Overwriting pi.cu


In [ ]:
!nvcc pi.cu -o pi

pi.cu(32): warning #177-D: variable "sum" was declared but never referenced



In [ ]:
!./pi 

Pi is: 0
